# GROUPING

After preparing the data for a more exhaustive analysis, the last step is to separate the different countries into group with common characteristics. This is necessary as an aggregate study of all countries would mean erasing the differences between them and missing important conclusions, making our study too shallow.

Grouping them by similar characteristics will simplify our analysis, all while reducing the data loss and stressing common patterns among countries in the same group.

In [1]:
import os
import numpy as np
from scipy.stats import pearsonr, spearmanr
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
from Project.Utils.aggregate import aggregate


column_country = 'Country'
column_year = 'Year'
column_region = 'Region'

read_path = os.getcwd()
region_url = read_path + '/Databases/AuxiliarData/world-regions.csv'
df_url = read_path + '/Output/SilverDataframe.csv'

write_path = os.getcwd() + '/Output/'
country_folder = write_path + '/Country/'
region_folder = write_path + '/Region/'
aggregate_folder = region_folder + '/Aggregate/'



## CREATING DIRECTORIES
If the destination folders do not exist, create them.

In [2]:
if not os.path.exists(country_folder):
            os.makedirs(country_folder)
if not os.path.exists(region_folder):
            os.makedirs(region_folder)
if not os.path.exists(aggregate_folder):
            os.makedirs(aggregate_folder)

### READ THE FULL DATAFRAME AND THE REGION DATAFRAME

In [3]:
region_df = pd.read_csv(region_url)
df = pd.read_csv(df_url, index_col = ['Country', 'Year'])

print(region_df)
region_df.drop(columns=['Code', 'Year'], inplace = True)
region_df.rename(columns = {'World Region according to the World Bank': column_region, 'Entity': column_country}, inplace = True)
region_df.set_index(['Country', 'Region'], inplace = True)
print(region_df)

             Entity Code  Year World Region according to the World Bank
0       Afghanistan  AFG  2017                               South Asia
1           Albania  ALB  2017                  Europe and Central Asia
2           Algeria  DZA  2017             Middle East and North Africa
3    American Samoa  ASM  2017                    East Asia and Pacific
4           Andorra  AND  2017                  Europe and Central Asia
..              ...  ...   ...                                      ...
212       Venezuela  VEN  2017              Latin America and Caribbean
213         Vietnam  VNM  2017                    East Asia and Pacific
214           Yemen  YEM  2017             Middle East and North Africa
215          Zambia  ZMB  2017                       Sub-Saharan Africa
216        Zimbabwe  ZWE  2017                       Sub-Saharan Africa

[217 rows x 4 columns]
Empty DataFrame
Columns: []
Index: [(Afghanistan, South Asia), (Albania, Europe and Central Asia), (Algeria, Mid

## ADD THE PROPERTY 'REGION'
By merging both DataFrames, we will obtain a DataFrame similar to the original one with a new field for the index: `region`

In [4]:
print(df)
df = pd.merge(df, region_df, how = 'inner', left_index = True, right_index = True)
print(df)

                  AgriShareGDP  CreditToAgriFishForest  EmploymentRural  \
Country     Year                                                          
Afghanistan 2000      54.06300                     NaN              NaN   
            2001      54.06300                     NaN              NaN   
            2002      45.13440                     NaN              NaN   
            2003      41.90340                     NaN              NaN   
            2004      35.61280                     NaN              NaN   
...                        ...                     ...              ...   
Zimbabwe    2016       7.87399                     NaN              NaN   
            2017       8.34095                     NaN              NaN   
            2018       8.30469                     NaN              NaN   
            2019       8.17322                     NaN              NaN   
            2020      10.93630                     NaN              NaN   

                        

## EXPORT THE SEGREGATED DATAFRAMES
We will export the new DataFrame with the regions and two new kinds of DataFrame: one for every different country, and another for every region.

In [5]:
df.to_csv(write_path + '/GoldDataframe.csv')

country_list = set(df.index.get_level_values(0))
region_list = set(df.index.get_level_values(2))

for country in country_list:
    df.loc[df.index.get_level_values(0) == country].to_csv(country_folder + country + '.csv')

for region in region_list:
    df.loc[df.index.get_level_values(2) == region].to_csv(region_folder + region + '.csv')

# REGION AGGREGATES
In order to create an aggregated DataFrame with entries for the regions we are studying, we will have to estimate the values of its indicators: the summatory for absolute ones, and a weighted average based on population for the relative ones. The weight of the mean could also be established as the GDP of each country, or any other figure we consider representative enough of the country's importance in the calculus.

In [6]:
#Define which indicators should be not a weighted
abs_indicators = ['GDP', 'Population', 'TotalAgri']

#Wether aggregate from the regions aggregate (faster) or from the country raw data (slower but more precise)
fast_aggregate = True

aggregate_df = pd.DataFrame()

for region in region_list:
    #Create sub-DataFrame with only the countries for each region
    region_df = df.loc[df.index.get_level_values(column_region) == region]
    #Aggregate by the countries of said region
    aggregated_region_df = aggregate(region_df, aggregate_by = column_country, for_index = column_year, new_group_col_name = column_region, group_name = region, abs_indicators = abs_indicators)
    #Add the aggregated DataFrame to the DataFrame per aggregated regions
    aggregate_df = pd.concat([aggregate_df, aggregated_region_df])

#Obtain an aggregate for the world
if fast_aggregate:
    aggregate_df_world = aggregate(aggregate_df, column_region, column_year, new_group_col_name = column_region, group_name = 'World', abs_indicators = abs_indicators)
else:
    aggregate_df_world = aggregate(df, column_country, column_year, new_group_col_name = column_region, group_name = 'World', abs_indicators = abs_indicators)

#Export the aggregated DataFrame to a .csv file
aggregate_df.to_csv(write_path + 'AggregatedRegion_DataFrame.csv')

aggregate_df_world.to_csv(write_path + 'AggregatedWorld_DataFrame.csv')

In [7]:
import datetime

#Set True if you want to analyse the time for both methods. By default is False to save that time.
test_time = False

if test_time:
    #Count time for the fast method: aggregating from the aggregated regions.
    st_time = datetime.now()
    fast_aggregate_df = aggregate(aggregate_df, column_region, column_year, new_group_col_name = column_region, group_name = 'World', abs_indicators = abs_indicators)
    f_time = datetime.now()
    fast_time = f_time - st_time
    print('Fast aggregation time: ' + fast_time.total_seconds())

    #Count time for the most precise method: aggregating from the raw data per country.
    st_time = datetime.now()
    precise_aggregate_df = aggregate(df, column_country, column_year, new_group_col_name = column_region, group_name = 'World',abs_indicators = abs_indicators)
    f_time = datetime.now()
    precise_time = f_time - st_time
    print('Precisse aggregation time: ' + precise_time.total_seconds())

    #Print the results to compare.
    print(fast_aggregate_df)
    print(precise_aggregate_df)

    #Export the results to .csv files if a more detailed analysis is needed.
    fast_aggregate_df.to_csv(region_folder + 'World_Fast.csv')
    precise_aggregate_df.to_csv(region_folder + 'World.csv')
else:
    print('No aggregation time analysed. If you want to study it, set the test_time variable to True and execute again')

No aggregation time analysed. If you want to study it, set the test_time variable to True and execute again


### Variables that can be changed
The p-value can be changed to a desired value. For the default value is recommnended 0.05. If you desire a higher confidence level lower this variable. This variable reffers directly to significance level, but for porpouses of clarification is set to this name:

We do the %store in order to pass this variable to other notebooks.

In [8]:
PVALUE_VAR = 0.05

%store PVALUE_VAR

Stored 'PVALUE_VAR' (float)


## Correlation dataframe.
This dataframe is the main piece of the notebook. Consists in generating for every country the correlation matrix for it and saving only the correlation value of the different variables with the GDP. 
This codeblock also calculates the p-value of Spearman and Pearson, if the value > PVALUE_VAR the correlation will be deleted due to not having statiscal significance.

Later on is concatenated and generates the following result:

In [9]:
#One dataframe per country

write_path = os.getcwd() + '/Output/'

col_country = 'Country'
col_year = 'Year'
col_region = 'Region'
col_gdp = 'GDP'

df= pd.read_csv (write_path + 'GoldDataframe.csv')
corr_df = pd.DataFrame()
corr_df.index.names = [col_country]

#List all the countries, none repeated
countries = set(df[col_country].to_list())

country_dict = {}
corr_dict = {}

for country in countries:
    #Get the DataFrame for a given country
    country_df = df.loc[df[col_country] == country]

    #Correlation matrix for that country
    country_corr_df = country_df.corr()

    #Significance for the correlations
    pval = country_df.corr(method = lambda x, y: pearsonr(x, y)[1]) - np.eye(*country_corr_df.shape)
    p = pval.applymap(lambda x: 1 if x < PVALUE_VAR else np.NaN)
    country_corr_df = country_corr_df * p

    #Significance for the correlations
    pval = country_df.corr(method = lambda x, y: spearmanr(x, y)[1]) - np.eye(*country_corr_df.shape)
    p = pval.applymap(lambda x: 1 if x < PVALUE_VAR else np.NaN)
    country_corr_df = country_corr_df * p

    #Trim it into a single row
    country_corr_df = country_corr_df.rename(columns = {col_gdp: country}).drop(index = [col_year, col_gdp])

    #Add the row to a new DataFrame with the correlations for each country
    corr_df = pd.concat([corr_df, country_corr_df[country]], axis = 1)

#Transpose the resulting DataFrame to have the desired format, save it and show it
corr_df = corr_df.transpose()
corr_df


,AgriShareGDP,CreditToAgriFishForest,EmploymentRural,TotalAgri,% Soldiers,Birth Rate,Death Rate,Homicides,Life Expectancy,Maternal Death Risk,...,% Population Growth,% Rural Population,Civil Liberties,Freedom of Expression,% Healthcare Investment,% Employment Industry,% Education Expenditure,% Men Employment,% Women Employment,Population
United Kingdom,-0.716390,NaN,NaN,0.865545,-0.841970,NaN,NaN,-0.741227,0.787113,NaN,...,0.789781,-0.737243,-0.588531,NaN,0.699031,-0.754821,0.468762,NaN,0.626019,0.701265
Bhutan,-0.802598,0.769226,NaN,0.921324,NaN,-0.951992,-0.866489,NaN,0.970373,-0.926428,...,-0.738949,-0.986241,0.846118,0.788449,NaN,NaN,NaN,NaN,NaN,0.985247
Cyprus,-0.919904,NaN,NaN,0.823334,NaN,-0.720753,NaN,NaN,0.665490,-0.691913,...,-0.550877,0.735158,0.831190,NaN,NaN,-0.520862,NaN,NaN,0.490699,0.788962
Montenegro,-0.925047,NaN,NaN,NaN,NaN,-0.748720,0.721335,-0.575695,0.936527,-0.961298,...,-0.635600,-0.934981,NaN,0.825413,NaN,-0.895129,NaN,NaN,NaN,0.951388
Cambodia,-0.818939,0.929465,NaN,0.990134,-0.597354,-0.982020,-0.842035,NaN,0.923128,-0.901626,...,-0.591579,-0.992325,-0.928227,-0.949780,-0.552843,0.859220,NaN,NaN,NaN,0.986496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Madagascar,-0.856914,NaN,NaN,0.645772,-0.892131,-0.962947,-0.957361,NaN,0.955664,-0.963204,...,-0.967226,-0.944561,NaN,NaN,NaN,0.718184,0.500264,NaN,NaN,0.943587
Saudi Arabia,-0.895421,NaN,NaN,0.914143,-0.940812,-0.920503,-0.854363,NaN,0.938614,-0.966087,...,NaN,-0.937296,0.854397,NaN,NaN,0.816189,NaN,NaN,NaN,0.937013
Sao Tome and Principe,-0.729275,NaN,NaN,NaN,NaN,-0.990529,-0.915989,NaN,0.947938,-0.938216,...,NaN,-0.975995,0.923671,0.603643,NaN,-0.876400,0.688839,NaN,NaN,0.985990
Ethiopia,-0.742794,NaN,NaN,0.856816,-0.660334,-0.910035,-0.878882,NaN,0.914411,-0.889476,...,-0.845126,-0.989688,NaN,NaN,-0.470976,-0.801164,NaN,NaN,NaN,0.978491


## Cleaned GoldDataframe 
Before the correlation dataframe was generated and it detected all the correlations with a high p-value. To perform all the following analysis a clean GoldDataframe is needed. It simply detects if the correlation for the indicator and country is Nan and procedes to delete the whole column indicator for the country.

In [10]:

df = df.copy()
for country in countries:
    for ind in corr_df.columns:
        if np.isnan(corr_df[ind][country]):
            df.loc[df[col_country] == country, [ind]] = np.NaN

df.set_index([col_country, col_region, col_year]).to_csv(write_path + 'GoldDataframe_significant.csv')
df.set_index([col_country, col_region, col_year])

AgriShareGDP  CreditToAgriFishForest  \
Country     Region             Year                                         
Afghanistan South Asia         2000      54.06300                     NaN   
                               2001      54.06300                     NaN   
                               2002      45.13440                     NaN   
                               2003      41.90340                     NaN   
                               2004      35.61280                     NaN   
...                                           ...                     ...   
Zimbabwe    Sub-Saharan Africa 2016       7.87399                     NaN   
                               2017       8.34095                     NaN   
                               2018       8.30469                     NaN   
                               2019       8.17322                     NaN   
                               2020      10.93630                     NaN   

                                     EmploymentRural           GDP  TotalAgri  \
Country     Region             Year                                             
Afghanistan South Asia         2000              NaN   3342.034168        NaN   
                               2001              NaN   3598.470576        NaN   
                               2002              NaN   4141.523943        NaN   
                               2003              NaN   4729.042179        NaN   
                               2004              NaN   5388.482107        NaN   
...                                              ...           ...        ...   
Zimbabwe    Sub-Saharan Africa 2016              NaN  20548.678073        NaN   
                               2017              NaN  22040.902301        NaN   
                               2018              NaN  24311.560545        NaN   
                               2019              NaN  21935.075306        NaN   
                               2020              NaN  21786.743637        NaN   

                                     % Soldiers  Birth Rate  Death Rate  \
Country     Region             Year                                       
Afghanistan South Asia         2000         NaN      48.021      11.718   
                               2001         NaN      47.505      11.387   
                               2002         NaN      46.901      11.048   
                               2003         NaN      46.231      10.704   
                               2004         NaN      45.507      10.356   
...                                         ...         ...         ...   
Zimbabwe    Sub-Saharan Africa 2016     0.75975         NaN       8.286   
                               2017     0.75072         NaN       8.044   
                               2018     0.73821         NaN       7.883   
                               2019     0.73821         NaN       7.773   
                               2020     0.73821         NaN       7.692   

                                     Homicides  Life Expectancy  ...  \
Country     Region             Year                              ...   
Afghanistan South Asia         2000        NaN           55.841  ...   
                               2001        NaN           56.308  ...   
                               2002        NaN           56.784  ...   
                               2003        NaN           57.271  ...   
                               2004        NaN           57.772  ...   
...                                        ...              ...  ...   
Zimbabwe    Sub-Saharan Africa 2016        NaN           60.294  ...   
                               2017        NaN           60.812  ...   
                               2018        NaN           61.195  ...   
                               2019        NaN           61.490  ...   
                               2020        NaN           61.738  ...   

                                     % Population Growth  % Rural Po

## Saving Correlation Dataframe Pearson

Use the same code as before, but this time save the correlation dataframe.

In [11]:
#One dataframe per country

write_path = os.getcwd() + '/Output/'

col_country = 'Country'
col_year = 'Year'
col_region = 'Region'
col_gdp = 'GDP'

df= pd.read_csv (write_path + 'GoldDataframe_significant.csv')
corr_df = pd.DataFrame()
corr_df.index.names = [col_country]

#List all the countries, none repeated
countries = set(df[col_country].to_list())

country_dict = {}
corr_dict = {}

for country in countries:
    #Get the DataFrame for a given country
    country_df = df.loc[df[col_country] == country]

    #Correlation matrix for that country
    country_corr_df = country_df.corr()

    #Trim it into a single row
    country_corr_df = country_corr_df.rename(columns = {col_gdp: country}).drop(index = [col_year, col_gdp])

    #Add the row to a new DataFrame with the correlations for each country
    corr_df = pd.concat([corr_df, country_corr_df[country]], axis = 1)

#Transpose the resulting DataFrame to have the desired format, save it and show it
corr_df = corr_df.transpose()
corr_df.to_csv(os.getcwd()+'/Output/Corr_DF_pearson.csv')
corr_df

,AgriShareGDP,CreditToAgriFishForest,EmploymentRural,TotalAgri,% Soldiers,Birth Rate,Death Rate,Homicides,Life Expectancy,Maternal Death Risk,...,% Population Growth,% Rural Population,Civil Liberties,Freedom of Expression,% Healthcare Investment,% Employment Industry,% Education Expenditure,% Men Employment,% Women Employment,Population
United Kingdom,-0.716390,NaN,NaN,0.865545,-0.841970,NaN,NaN,-0.741227,0.787113,NaN,...,0.789781,-0.737243,-0.588531,NaN,0.699031,-0.754821,0.468762,NaN,0.626019,0.701265
Bhutan,-0.802598,0.769226,NaN,0.921324,NaN,-0.951992,-0.866489,NaN,0.970373,-0.926428,...,-0.738949,-0.986241,0.846118,0.788449,NaN,NaN,NaN,NaN,NaN,0.985247
Cyprus,-0.919904,NaN,NaN,0.823334,NaN,-0.720753,NaN,NaN,0.665490,-0.691913,...,-0.550877,0.735158,0.831190,NaN,NaN,-0.520862,NaN,NaN,0.490699,0.788962
Montenegro,-0.925047,NaN,NaN,NaN,NaN,-0.748720,0.721335,-0.575695,0.936527,-0.961298,...,-0.635600,-0.934981,NaN,0.825413,NaN,-0.895129,NaN,NaN,NaN,0.951388
Cambodia,-0.818939,0.929465,NaN,0.990134,-0.597354,-0.982020,-0.842035,NaN,0.923128,-0.901626,...,-0.591579,-0.992325,-0.928227,-0.949780,-0.552843,0.859220,NaN,NaN,NaN,0.986496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Madagascar,-0.856914,NaN,NaN,0.645772,-0.892131,-0.962947,-0.957361,NaN,0.955664,-0.963204,...,-0.967226,-0.944561,NaN,NaN,NaN,0.718184,0.500264,NaN,NaN,0.943587
Saudi Arabia,-0.895421,NaN,NaN,0.914143,-0.940812,-0.920503,-0.854363,NaN,0.938614,-0.966087,...,NaN,-0.937296,0.854397,NaN,NaN,0.816189,NaN,NaN,NaN,0.937013
Sao Tome and Principe,-0.729275,NaN,NaN,NaN,NaN,-0.990529,-0.915989,NaN,0.947938,-0.938216,...,NaN,-0.975995,0.923671,0.603643,NaN,-0.876400,0.688839,NaN,NaN,0.985990
Ethiopia,-0.742794,NaN,NaN,0.856816,-0.660334,-0.910035,-0.878882,NaN,0.914411,-0.889476,...,-0.845126,-0.989688,NaN,NaN,-0.470976,-0.801164,NaN,NaN,NaN,0.978491


Saving the correlation dataframe using spearman.

In [12]:
#One dataframe per country

write_path = os.getcwd() + '/Output/'

col_country = 'Country'
col_year = 'Year'
col_region = 'Region'
col_gdp = 'GDP'

df= pd.read_csv (write_path + 'GoldDataframe_significant.csv')
corr_df = pd.DataFrame()
corr_df.index.names = [col_country]

#List all the countries, none repeated
countries = set(df[col_country].to_list())

country_dict = {}
corr_dict = {}

for country in countries:
    #Get the DataFrame for a given country
    country_df = df.loc[df[col_country] == country]

    #Correlation matrix for that country
    country_corr_df = country_df.corr(method='spearman')

    #Trim it into a single row
    country_corr_df = country_corr_df.rename(columns = {col_gdp: country}).drop(index = [col_year, col_gdp])

    #Add the row to a new DataFrame with the correlations for each country
    corr_df = pd.concat([corr_df, country_corr_df[country]], axis = 1)

#Transpose the resulting DataFrame to have the desired format, save it and show it
corr_df = corr_df.transpose()
corr_df.to_csv(os.getcwd()+'/Output/Corr_DF_spearman.csv')
corr_df

,AgriShareGDP,CreditToAgriFishForest,EmploymentRural,TotalAgri,% Soldiers,Birth Rate,Death Rate,Homicides,Life Expectancy,Maternal Death Risk,...,% Population Growth,% Rural Population,Civil Liberties,Freedom of Expression,% Healthcare Investment,% Employment Industry,% Education Expenditure,% Men Employment,% Women Employment,Population
United Kingdom,-0.548943,NaN,NaN,0.781535,-0.785156,NaN,NaN,-0.53125,0.695285,NaN,...,0.547465,-0.676203,-0.702659,NaN,0.664716,-0.651665,0.472930,NaN,0.670166,0.676203
Bhutan,-0.691783,0.604289,NaN,0.893506,NaN,-0.993506,-0.971095,NaN,0.993506,-0.992839,...,-0.589152,-0.993506,0.784252,0.621587,NaN,NaN,NaN,NaN,NaN,0.993506
Cyprus,-0.564469,NaN,NaN,0.750244,NaN,-0.580058,NaN,NaN,0.580519,-0.566778,...,-0.450649,0.584416,0.481672,NaN,NaN,-0.504237,NaN,NaN,0.444454,0.580519
Montenegro,-0.802209,NaN,NaN,NaN,NaN,-0.733382,0.732283,-0.51172,0.848327,-0.905542,...,-0.571429,-0.901299,NaN,0.747359,NaN,-0.831275,NaN,NaN,NaN,0.897403
Cambodia,-0.672296,0.936930,NaN,0.992208,-0.993499,-0.998701,-0.998701,NaN,0.998701,-0.996748,...,-0.587013,-0.998701,-0.880553,-0.984348,-0.522417,0.985022,NaN,NaN,NaN,0.998701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Madagascar,-0.915232,NaN,NaN,0.533766,-0.957088,-0.957143,-0.957143,NaN,0.957143,-0.961569,...,-0.957143,-0.957143,NaN,NaN,NaN,0.738766,0.514955,NaN,NaN,0.957143
Saudi Arabia,-0.921728,NaN,NaN,0.883117,-0.919377,-0.918182,-0.800000,NaN,0.918182,-0.918572,...,NaN,-0.918182,0.747887,NaN,NaN,0.738766,NaN,NaN,NaN,0.918182
Sao Tome and Principe,-0.616434,NaN,NaN,NaN,NaN,-0.994805,-0.993506,NaN,0.993506,-0.990233,...,NaN,-0.993506,0.946628,0.698361,NaN,-0.930298,0.691409,NaN,NaN,0.993506
Ethiopia,-0.598246,NaN,NaN,0.902597,-0.988297,-0.993506,-0.993506,NaN,0.993506,-0.990233,...,-0.777634,-0.993506,NaN,NaN,-0.704890,-0.792187,NaN,NaN,NaN,0.993506
